In [29]:
from torchvision import datasets, transforms as Transforms

class ContrastiveLearningViewGenerator(object):
    """Take two random crops of one image as the query and key."""

    def __init__(self, base_transform, n_views=2):
        self.base_transform = base_transform
        self.n_views = n_views

    def __call__(self, x):
        return [self.base_transform(x) for i in range(self.n_views)]

def get_moco_transforms(net_type='encoder_train', resize=96, crop=96):
    transform_list = []
    if net_type == 'encoder_train':
        transform_list += [Transforms.RandomResizedCrop(size=crop),
                            Transforms.ColorJitter(0.1, 0.1, 0.1),
                            Transforms.RandomHorizontalFlip(),
                            Transforms.RandomGrayscale()]
    elif net_type == 'classifier_train':
        transform_list += [Transforms.Resize(size=resize),
                            Transforms.RandomCrop(size=crop),
                            Transforms.RandomHorizontalFlip()]
    elif net_type == 'classifier_test':
        transform_list += [Transforms.Resize(size=resize),
                            Transforms.CenterCrop(size=crop)]

    transform_list += [Transforms.ToTensor(),
                        Transforms.Normalize(mean=(0.5, 0.5, 0.5),
                                            std=(0.5, 0.5, 0.5))]

    transform = Transforms.Compose(transform_list)
    return transform


stl10_dest_pth = r'E:\YuChe\MS\structured_ML\SimCLR\datasets'

# dset = datasets.STL10(root=stl10_dest_pth, split='train', download=False)
dset = datasets.STL10(root=stl10_dest_pth, split='train', transform=ContrastiveLearningViewGenerator(
                                                              get_moco_transforms(96)), download=False)


In [5]:
import torch

tensor = torch.tensor([0,1], dtype=torch.long)
tensor = torch.nn.functional.one_hot(tensor.type(torch.int64), 3).type(torch.float64)
tensor.size(1)


3

### feature eval

In [1]:
import os

import torch
import torch.nn as nn

import model
from dataloader import data_loader
from metric import eval_metric, accuracy, f1_score

In [3]:
dev = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 1024
exp_version = 'v3'

## Load the model
Encoder_weight_path = os.path.join('./output/Stl-10', f'bz{batch_size}', exp_version, 'weight', 'ckpt_199.pkl')
Encoder_ckpt = torch.load(Encoder_weight_path)
Linear_weight_path = os.path.join('./output/Stl-10', f'bz{batch_size}', exp_version, 'eval/weight', 'ckpt_99.pkl') # TODO: change to the best weight
Linear_ckpt = torch.load(Linear_weight_path)

encoder = nn.DataParallel(model.Resnet50(dim=128)).to(dev)
encoder.load_state_dict(Encoder_ckpt['encoder'], strict=False)
feature_extractor = nn.Sequential(*list(encoder.module.resnet.children())[:-1]) # feature extractor from encoder

linear = nn.Linear(2048, 10).to(dev) # linear classifier
linear.load_state_dict(Linear_ckpt)
print('Model loaded')


Model loaded


In [4]:
## load the data
tst_dloader, tst_dlen = data_loader(dataset_root='', # dataset root is not needed
                                    resize=84, 
                                    crop=64,
                                    batch_size=100,
                                    num_workers=16,
                                    type='classifier_test')
print('Data loaded')

Data loaded


In [5]:

def cm_metric(output, target, topk=(1,)):
    """
        output: model output, shape: (B, num_classes)
        target: ground truth label, shape: (B,)
    Computes the top_k accuracy and f1_score for each class."""
    output = output.to('cpu')
    target = target.to('cpu')
    acc = accuracy(output, target, topk)
    f1 = f1_score(output, target)
    return acc, f1

sampling_times = 5
history = {'acc': [], 'f1': []}
with torch.no_grad():
    for _ in range(sampling_times):
        for idx, (img, label) in enumerate(tst_dloader):
            print(f'Batch {idx+1}/{tst_dlen/100:.0f}', end='\r')
            img = img.to(dev)
            label = label.to(dev)
            feature = feature_extractor(img)
            score = linear(feature.view(feature.size(0), feature.size(1)))
            
            acc, f1 = cm_metric(score, label, topk=(1, 5))
            history['acc'].append(acc)
            history['f1'].append(f1)

In [7]:
from torch import tensor
Acc, F1_score = torch.stack([tensor(acc) for acc in history['acc']]), torch.stack([tensor(f1) for f1 in history['f1']])

print('- Top-1 Acc: ${:.2f} \pm {:.2f}$'.format(Acc[:, 0].mean().item(), Acc[:, 0].std().item()))
print('- Top-5 Acc: ${:.2f} \pm {:.2f}$'.format(Acc[:, 1].mean().item(), Acc[:, 1].std().item()))

for cls_id in range(F1_score.size(1)):
    # print('Class {}: {:.2f} +/- {:.2f}'.format(cls_id, F1_score[:, cls_id].mean().item()*100, F1_score[:, cls_id].std().item()*100))
    print('- F1-{}: ${:.2f} \pm {:.2f}$'.format(cls_id, F1_score[:, cls_id].mean().item()*100, F1_score[:, cls_id].std().item()*100))


- Top-1 Acc: $62.84 \pm 5.14$
- Top-5 Acc: $96.97 \pm 1.78$
- F1-0: $72.75 \pm 12.89$
- F1-1: $57.51 \pm 13.96$
- F1-2: $72.82 \pm 11.69$
- F1-3: $46.46 \pm 14.44$
- F1-4: $57.35 \pm 12.90$
- F1-5: $46.04 \pm 14.76$
- F1-6: $66.54 \pm 12.85$
- F1-7: $57.50 \pm 13.95$
- F1-8: $77.09 \pm 10.66$
- F1-9: $64.29 \pm 12.86$


In [77]:
x_k = torch.arange(0, 9)
idx = torch.randperm(x_k.size(0))
x_k = x_k[idx]
inv_idx = torch.argsort(idx)

x_k[inv_idx]

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])